# Praca domowa nr 5

Budowanie zbioru modeli Rashomon.

Modele są zbudowany na podstawie preprocessingu przedstawionego w artykule: https://academic.oup.com/jamiaopen/article/1/1/87/5032901. 

Kod do artykułu dostępny jest pod linkiem: https://github.com/illidanlab/urgent-care-comparative

Zadanie: problem klasyfikacji, predykcja śmiertelności na podstawie przedstawienia danych w postaci *X48* (wg. artykułu powyżej).

### Biblioteki

In [1]:
import numpy as np
import pandas as pd

import pickle
import os.path

import xgboost as xgb

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import auc as auc_score
from sklearn.utils import shuffle

ModuleNotFoundError: No module named 'xgboost'

### Załadowanie danych po preprocessingu

In [ ]:
X = np.load("X48.npy")

In [ ]:
with open('y.npy', 'rb') as f:
    labels = pickle.load(f)
    
task = [yy[0] for yy in labels]
y = np.array(task)

### Generowanie próbek do kroswalidacji

Przy modelowaniu skorzystamy z pięciokrotnej kroswalidacji - w celu zapewnienia reprodukowalności, indeksy użytych próbek można wczytać z pliku:

In [ ]:
def get_cv_samples_indexes(X, y):
    if os.path.isfile('samples.npy'):
        return np.load("samples.npy", allow_pickle = True)
    else:
        tab = []
        skf = StratifiedKFold(n_splits = 5)
        
        for train_index, test_index in skf.split(X, y):
            tab.append((train_index, test_index))
            
        with open('samples.npy', 'wb') as f:
            pickle.dump(tab, f)
            
        return tab

In [ ]:
cv_tab = get_cv_samples_indexes(X, y)

### Obiekty - model, random search, siatka hiperparametrów

In [ ]:
model = xgb.XGBClassifier(objective='binary:logistic', n_jobs = -1, eval_metric = 'auc', use_label_encoder = False, seed = 123)

Zakres hiperparametrów wzorowany artykułem (tabela 1): https://jmlr.org/papers/volume20/18-444/18-444.pdf

Dokumentacja parametrów: https://xgboost.readthedocs.io/en/latest/parameter.html

In [ ]:
hyperparameters =  {
    'learning_rate' : 2 ** np.linspace(-10, 0, num = 20),
    'subsample' : np.linspace(0.1, 1, num = 20),
    'booster' : ['gbtree', 'dart'],
    'max_depth' : list(range(1, 15 + 1)),
    'min_child_weight' : 2 ** np.linspace(0, 7, num = 20),
    'colsample_bytree' : np.linspace(0.001, 1, num = 20),
    'colsample_bylevel' : np.linspace(0.001, 1, num = 20),
    'lambda' : 2 ** np.linspace(-10, 10, num = 20),
    'alpha' : 2 ** np.linspace(-10, 10, num = 20),
    'n_estimators' : list(range(30, 740, 50))
}

In [ ]:
class cross_val_gen:
    def __init__(self, cv_tab):
        self.n_splits = 5
        self.cv_tab = cv_tab

    def split(self, X, y, groups=None):
        for train_index, test_index in cv_tab:
            yield train_index, test_index 

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [ ]:
number_of_models = 50

In [ ]:
cv_search_obj = RandomizedSearchCV(estimator = model, param_distributions = hyperparameters, n_iter = number_of_models, 
                                   scoring = 'roc_auc', cv = cross_val_gen(cv_tab), return_train_score = True, verbose = 2)

### Modelowanie

In [ ]:
search = cv_search_obj.fit(X, y)

### Ramka danych wynikowych

In [ ]:
results = pd.DataFrame(search.cv_results_)

In [ ]:
results.columns

In [ ]:
results.head()

In [ ]:
with open('results.npy', 'wb') as f:
    pickle.dump(results, f)

In [ ]:
results.to_csv("results.csv")